훈련하는 도중이나 훈련이 끝난 후에 모델을 저장할 수 있다. 모델을 중지된 지점부터 다시 훈련할 수 있어 한 번에 오랫동안 훈련하지 않아도 된다. 또 모델을 저장하면 다른 사람에게 공유할 수 있고 작업을 재현할 수 있다. 연구한 모델과 기법을 공개할 때 많으 머신러닝 기술자들이 다음과 같은 것들을 제공한다.

> 모델을 만드는 코드

> 모델의 훈련된 가중치 또는 파라미터

# 설정

## 설치와 임포트



In [ ]:
pip install -q pyyaml h5py  # HDF5 포맷으로 모델을 저장하기 위해서 필요합니다

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

## 예제 데이터셋 받기

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## 모델 정의

In [ ]:
# 간단한 Sequential 모델을 정의합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# 모델 객체를 만듭니다
model = create_model()

# 모델 구조를 출력합니다
model.summary()

#  훈련하는 동안 체크포인트 저장하기

훈련 중간과 훈련 마지막에 체크 포인트를 자동으로 저장하도록 하는 것이 많이 사용하는 방법이다. 다시 훈련하지 않고 모델을 재사용하거나 훈련 과정이 중지된 경우 이어서 훈련할 수 있다.

## 체큰포인트 콜백 사용하기

훈련하는 동안 가중치를 저장히기 위해 ModelCheckpoint 콜백을 만든다.

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

In [ ]:
ls {checkpoint_dir}

훈련하지 않은 새로운 모델을 만들어 보자. 가중치만 복원할 땐 원본 모델과 동일한 구조로 모델을 만들어야 한다. 여기서 동일한 구조로 모델을 만들었으므로 다른 객체이지만 가중치를 공유할 수 있다.

훈련하지 않은 새모델을 만들고 테스트 세트에서 평가해보자. 훈련되지 않은 모델의 성능은 무작위로 선택하는 정도의 수준이다.

In [ ]:
# 기본 모델 객체를 만듭니다
model = create_model()

# 모델을 평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

체크 포인트에서 가중치를 로드하고 다시 평가해보자

In [ ]:
# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

## 체크 포인트 콜백 매개변수

이 콜백 함수는 몇 가지 매개변수를 제공한다. 체크 포인트 이름을 고유하게 만들거나 체크 포인트 주기를 조정할 수 있다.

새로운 모델을 훈련하고 다섯번의 에포크마다 고유한 이름으로 체크 포인트를 저장보자.

In [ ]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 새로운 모델 객체를 만듭니다
model = create_model()

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)

In [ ]:
ls {checkpoint_dir}

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

In [ ]:
# 새로운 모델 객체를 만듭니다
model = create_model()

# 이전에 저장한 가중치를 로드합니다
model.load_weights(latest)

# 모델을 재평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

# 수동으로 가중치 저장하기
수동으로 가중치를 저장하는 것도 쉽다. Model.save_weights 메서드를 사용한다. ckpt 확장자를 가진 텐서플로 체크포인트 포맷을 사용한다. 

In [ ]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 새로운 모델 객체를 만듭니다
model = create_model()

# 가중치를 복원합니다
model.load_weights('./checkpoints/my_checkpoint')

# 모델을 평가합니다
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

# 전체 모델 저장하기

model.save 메서드를 호출하여 모델의 구조, 가중치, 훈련 설정을 하나의 파일/폴더에 저장한다. 모델을 저장하기 때문에 원본 파이썬 코드가 없어도 사용할 수 있다. 옵티마이저 상태가 복원되므로 정확히 중지한 시점에서 다시 훈련을 시작할 수 있다.

두 개의 포맷(SavedMdoel과 HDF5)으로 모델을 저장할 수 있다. 텐서플로의 SavedModel 포맷은 기본 파일 포맷이다. 하지만 HDF5 포맷으로 저장할 수도 있다. 

## SavedModel 포맷

SavedModel 포맷은 모델을 직렬화하는 다른 방법이다. 이 포맷으로 저장한 모델은 tf.keras.models.load_model로 복원할 수 있고 텐서플로 서빙과 호환된다.

In [ ]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조를 확인합니다
new_model.summary()

In [ ]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

## HDF5 파일로 저장하기

In [ ]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

In [ ]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model('my_model.h5')

# 모델 구조를 출력합니다
new_model.summary()

In [ ]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

케라스는 모델 구조를 파악하여 저장한다.

>가중치 값

>모델 구조

> 모델의 훈련 설정(컴파일할 떄 전달한 값)

>옵티마이저와 필요하면 옵티마이저의 상태(중지된 지점부터 다시 시작할 수 있다)

체크 포인트가 호환되지 않기 때문에 케라스는 v1.x 옵티마이저를 저장할 수 없다. v1.x 옵티마이저를 사용하려면 로드한 후 모델을 다시 컴파일해야한다. 따라서 옵티마이저의 상태를 잃게 된다.
